# Chatbot Neo4j Knowledge graph



In [ ]:
!pip install neo4j-driver
!pip install sentence-transformers

In [ ]:
#!pip install pyngrok
!pip install pyngrok==4.1.1
!pip install flask_ngrok
!pip install line-bot-sdk
!ngrok authtoken 2VdtRNXwme17FSHyeSwYc31YgbE_5VuchMnfVKeXRsw7GDdVE

เชื่อมต่อกับ google sheet

In [ ]:
!pip install gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
sheet_id = '1pc748jmYo1zkjwCgTequcixAUobVRxK7li5nc8EHwpA'
worksheet_name = "sheet1"

In [ ]:
import gspread

In [ ]:
from google.auth import default

In [ ]:
creds, _=default()

In [ ]:
gc = gspread.authorize(creds)

In [ ]:
sh = gc.open_by_key(sheet_id)

In [ ]:
sh.worksheets()

[<Worksheet 'sheet1' id:0>]

In [ ]:
worksheet = sh.worksheet(worksheet_name)

In [ ]:
worksheet.get_all_records()

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


#Connection Neo4j

In [ ]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "neo4j+s://e6edf238.databases.neo4j.io",
  auth=basic_auth("neo4j", "MY3uLzRnkio9nGTuz5D_mnCe7qyHeETw7hS_UETt0Pg"))

In [ ]:
#from pythainlp import word_tokenize

##Build Entities corpus

In [ ]:
cypher_query = '''
MATCH (n) return n.name as name;
'''
Entity_corpus = []
with driver.session() as session:
    results = session.run(cypher_query)
    for record in results:
        Entity_corpus.append(record['name'])
    # using set() to remove duplicated from list
    Entity_corpus = list(set(Entity_corpus))
print(Entity_corpus)

cypher_query = '''
MATCH (n)-[r]-(m) return type(r) as name;
'''
Relation_corpus = []
with driver.session() as session:
    results = session.run(cypher_query)
    for record in results:
        Relation_corpus.append(record['name'])
    # using set() to remove duplicated from list
    Relation_corpus = list(set(Relation_corpus))
print(Relation_corpus)

cypher_query = '''
MATCH (n:ทักทาย) return n.hello as name;
'''
greeting_corpus = []
with driver.session() as session:
    results = session.run(cypher_query)
    for record in results:
        greeting_corpus.append(record['name'])
    # using set() to remove duplicated from list
    greeting_corpus = list(set(greeting_corpus))
print(Relation_corpus)

['การเลือกรับประทานอาหาร', 'หลงทางในเส้นทางที่ใช้อยู่เป็นประจำ', 'กระเจี๊ยบแดง', 'ตกขาวผิดปกติ', 'โรคหลอดเลือดแดงใหญ่โป่งพอง', 'เชื้อชาติแอฟริกัน-อเมริกัน', 'ชาตามปลายนิ้วมือและนิ้วเท้า', 'โรคตา', 'ปัสสาวะบ่อยขึ้นหิวน้ำมากขึ้น', 'การรับประทานยาเม็ดคุมกำเนิด', 'อาเจียนหรือคลื่นไส้', 'โรคหลอดเลือดหัวใจ', 'เจ็บหน้าอก', 'พันธุกรรม', 'หลีกเลี่ยงการบริโภคอาหารที่มีไขมันอิ่มตัวสูงและอาหารรสเค็มจัด', 'ภาวะสมองขาดเลือดชั่วขณะ', None, 'น้ำตาลในเลือดสูง', 'ออกกำลังกายเป็นประจำ', 'ปวดเมื่อยตามตัว', 'การติดเชื้อ', 'ควบคุมน้ำหนัก', 'เวียนศรีษะ/ปวดศรีษะ', 'กระเทียมสด', 'รับประทานผลไม้และผัก', 'พูดหรือถามเรื่องเดิมที่เพิ่งเกิดขึ้นซ้ำๆ', 'การพบปะพูดคุยกับผู้อื่นบ่อยๆ', 'การทานอาหารที่มีไขมันมาก', 'ไอเรื้อรัง', 'โรคตับแข็ง', 'ภาวะขาดสารอาหาร', 'โรคเบาหวาน', 'การได้รับบาดเจ็บที่ศีรษะอย่างรุนแรง', 'ควบคุมระดับไขมันในเลือด', 'ภาวะฮีโมโครมาโตซิส', 'ไคร้เครือ', 'ภาวะหลอดเลือดสมองแตก', 'มะระขี้นก', 'ขี้เหล็ก', 'การขาดสารต้านทริปซินอัลฟ่าวัน', 'ขาดการออกกำลังกาย', 'อารมณ์แปรปรวน', 'ความผิดปกติของตับอ่อน', 'โรค

In [ ]:
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [ ]:
import numpy as np


In [ ]:


#greeting_corpus = ["สวัสดีครับ","ดีจ้า"]
corpus_vec = model.encode(Entity_corpus, convert_to_tensor=True,normalize_embeddings=True)
relation_vec = model.encode(Relation_corpus, convert_to_tensor=True,normalize_embeddings=True)
greeting_vec = model.encode(greeting_corpus, convert_to_tensor=True,normalize_embeddings=True)




def Search_entity(ask,user_id,line_bot_api,tk):
    #Compute embedding for both lists
    ask_vec = model.encode(ask, convert_to_tensor=True,normalize_embeddings=True)

    #Compute cosine-similarities
    entity_cosine_scores = util.cos_sim(corpus_vec, ask_vec)
    #ทำให้ทศนิยมมากขึ่้น
    #print(entity_cosine_scores)
    Entity_np = np.array(entity_cosine_scores)
    #print(Entity_np)
    Match_entity = Entity_corpus[np.argmax(entity_cosine_scores)]
    #Compute cosine-similarities
    relation_cosine_scores = util.cos_sim(relation_vec, ask_vec)
    Relation_np = np.array(relation_cosine_scores)
    Match_relation = Relation_corpus[np.argmax(Relation_np)]
    #greeting
    greeting_scores = util.cos_sim(greeting_vec, ask_vec)
    greeting_np = np.array(greeting_scores)
    Match_greeting = greeting_corpus[np.argmax(greeting_np)]
    ##############################################
    print("entity_score",Entity_np[np.argmax(Entity_np)])
    print("relation_score",Relation_np[np.argmax(Relation_np)])
    if greeting_np[np.argmax(greeting_np)] > 0.7 :
       n=4
       My_cypher = [f"MATCH (n)-[r]->(p) where n.hello ='{Match_greeting}' return n.name,n.description,n.arkname,n.image"]
       return greeting_reply(line_bot_api,tk,My_cypher)



    elif Relation_np[np.argmax(Relation_np)] > 0.65 and Relation_np[np.argmax(Relation_np)]>Entity_np[np.argmax(Entity_np)]:
        n=1
        My_cypher = [f"MATCH (n)-[r]->(p) WHERE type(r) ='{Match_relation}' RETURN n.name,r.name,p.name,p.description,type(r) limit 10"]

        return quick_reply_relationship(line_bot_api, tk, My_cypher)


    # Cypher sample
    elif Entity_np[np.argmax(Entity_np)] > 0.75:
        n=2
        My_cypher = [f"MATCH (n)-[r]-(p) WHERE p.name ='{Match_entity}'  RETURN p.name,p.description,p.image limit 10 "]
        return entity_respons(line_bot_api,tk,My_cypher)

    else:
        text_message = "ไม่มีข้อมูลในคลังความรู้ 😔"
        line_bot_api.reply_message(tk, TextSendMessage(text=text_message))



In [ ]:
def entity_respons(line_bot_api,tk,result):
    print("เข้า entity_Res")
    My_cypher = result
    quick_reply_button = QuickReplyButton(
        action=MessageAction(label="เมนู", text="เมนู")
    )
    quick_reply = QuickReply(
      items=[quick_reply_button]
    )

    with driver.session() as session:
            for Qcypher in My_cypher:
              result = session.run(Qcypher)
              for record in result:
                if(str(record['p.description']))=="None":
                  text_message = "ไม่มีข้อมูลในคลังความรู้ 🤯"
                  return line_bot_api.reply_message(tk, TextSendMessage(text=text_message))
                elif (str(record['p.description']))!="None" and (str(record['p.image']))!= "None":
                  print("image")
                  text_message = str(record['p.name'])+" "+str(record['p.description'])
                  url_pattern = "https://lh3.googleusercontent.com/d/"+str(record['p.image'])
                  image_url=ImageSendMessage(original_content_url=url_pattern, preview_image_url=url_pattern)
                  line_bot_api.reply_message(tk, [image_url, TextSendMessage(text=text_message,quick_reply=quick_reply)])
                else:
                  print("เข้า else")
                  text_message = str(record['p.name'])+" "+str(record['p.description'])
                  #url_pattern = "https://lh3.googleusercontent.com/d/"+str(record['p.image'])
                  #image_url=ImageSendMessage(original_content_url=url_pattern, preview_image_url=url_pattern)
                  line_bot_api.reply_message(tk, [TextSendMessage(text=text_message,quick_reply=quick_reply)])
                  #line_bot_api.reply_message(tk, TextSendMessage(text=text_message,quick_reply=quick_reply))



In [ ]:
Search_entity("โรคเบาหวาน","Ucc3e077cff6d7f39dc73f1ead7cc39e0","11","11")

In [ ]:
reply_msg("อาการโรคเบาหวาน","Ucc3e077cff6d7f39dc73f1ead7cc39e0","11","11")

NameError: ignored

In [ ]:
def check_user_id(line_bot_api,tk,user_id,msg):
# Open our new sheet and add some data.
  worksheet = gc.open_by_key('1pc748jmYo1zkjwCgTequcixAUobVRxK7li5nc8EHwpA').sheet1
  id_to_check = user_id
  all_values = worksheet.get_all_values()
  frow = None
  for row_number ,row_data in enumerate(worksheet.col_values(1),start=1):
    if row_data == user_id:
        frow = row_number
        print("frow", frow)


  # ตรวจสอบว่าข้อมูลมีอยู่แล้วหรือไม่
  data_exists = any([id_to_check in row for row in all_values])
  print("msg",msg)
  if data_exists:
      if (msg == "ประเมินโรคเบาหวาน" or worksheet.cell(frow, 2).value == "Evaluate" or msg == "ประเมินโรคความดันโลหิตสูง" or msg == "ประเมินโรคอ้วน" or msg == "ประเมินโรคตับแข็ง" or msg == "ประเมินโรคมะเร็ง" or msg == "ประเมินโรคหลอดเลือดสมอง" or msg == "ประเมินโรคสมองเสื่อม" or msg == "ประเมินโรคไตเรื้อรัง"):
            estimate(msg,user_id,line_bot_api,tk)
      elif (msg == "เมนู" or msg == "menu" or msg == "Menu"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "ประเมิน"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "สอบถาม"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "สมุนไพร"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "อาการ"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "สาเหตุ"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "วิธีป้องกัน"):
            return_message(line_bot_api,tk,user_id,msg)
      else:
        print("เรียก entity_if")
        Search_entity(msg,user_id,line_bot_api,tk)

  else:

      data_to_write = [user_id]
      col_a_length = len(worksheet.col_values(1))
      cell_to_update = f'A{col_a_length + 1}'  # ทำการหาเซลล์ที่ต้องการอัปเดต
      worksheet.update(cell_to_update, user_id)
      # หากข้อมูลยังไม่มีอยู่, ทำการเพิ่มข้อมูล
      #data_to_write = [user_id]
      #worksheet.append_row(data_to_write)
      #worksheet.insert_row(data_to_write,index=2)
      print("msg_check",msg)
      print("เพิ่มข้อมูลเรียบร้อยแล้ว")
      if  (msg == "ประเมินโรคเบาหวาน" or msg == "ประเมินโรคความดันโลหิตสูง" or msg == "ประเมินโรคอ้วน" or msg == "ประเมินโรคตับแข็ง" or msg == "ประเมินโรคมะเร็ง" or msg == "ประเมินโรคหลอดเลือดสมอง" or msg == "ประเมินโรคสมองเสื่อม" or msg == "ประเมินโรคไตเรื้อรัง"):
            estimate(msg,user_id,line_bot_api,tk)
      elif (msg == "เมนู" or msg == "menu" or msg == "Menu"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "ประเมิน"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "สอบถาม"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "สมุนไพร"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "อาการ"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "สาเหตุ"):
            return_message(line_bot_api,tk,user_id,msg)
      elif (msg == "วิธีป้องกัน"):
            return_message(line_bot_api,tk,user_id,msg)
      else:
        print("เรียก entity")
        Search_entity(msg,user_id,line_bot_api,tk)



In [ ]:
def estimate(msg,user_id,line_bot_api,tk):
    type_msg = ""
    n=2
    count_estimate = 0
    actual_status = '1'
    status_check = '1'
    send_loop = ""
    user_id_column = worksheet.col_values(1)
    cell = worksheet.find(user_id)
    if (user_id in user_id_column) and (worksheet.cell(cell.row, 2).value != "Evaluate"):
        # หา cell ที่มี user_id และอัปเดตค่า status ในคอลัมน์ B
        worksheet.update_cell(cell.row, 2, "Evaluate")
        worksheet.update_cell(cell.row, 3, actual_status)
        worksheet.update_cell(cell.row, 4, msg)
        worksheet.update_cell(cell.row, 17, status_check)
        type_msg = msg

    elif (user_id in user_id_column) and (worksheet.cell(cell.row, 2).value == "Evaluate"):
         type_msg = worksheet.cell(cell.row, 4).value
         actual_status = worksheet.cell(cell.row, 3).value
         status_check = worksheet.cell(cell.row, 17).value
         if (actual_status == '2'):
             if(msg == "ใช่" or msg =="เคย" or msg == "มี" or msg == "สูบ"):
                worksheet.update_cell(cell.row, 5, '1')
                worksheet.update_cell(cell.row, 17, '2')
             elif(msg == "ไม่ใช่" or msg =="ไม่เคย" or msg == "ไม่มี" or msg == "ไม่สูบ" or msg == "ไม่"):
                worksheet.update_cell(cell.row, 17, '2')
                worksheet.update_cell(cell.row, 5, '0')
             elif (msg == "ออกจากการประเมิน"):
                 worksheet.update_cell(cell.row, 2, " ")
                 worksheet.update_cell(cell.row, 3, " ")
                 worksheet.update_cell(cell.row, 4, " ")
                 worksheet.update_cell(cell.row, 17, " ")
                 actual_status = '5'
                 Carousel=response_Carousel()
                 line_bot_api.reply_message(tk,Carousel)
             else:
                send_loop = "loop"
                actual_status = status_check
         elif (actual_status == '3'):
              if(msg == "ใช่" or msg =="เคย" or msg == "มี" or msg == "สูบ"):
                worksheet.update_cell(cell.row, 6, '1')
                worksheet.update_cell(cell.row, 17, '3')
              elif(msg == "ไม่ใช่" or msg =="ไม่เคย" or msg == "ไม่มี" or msg == "ไม่สูบ"or msg == "ไม่"):
                worksheet.update_cell(cell.row, 17, '3')
                worksheet.update_cell(cell.row, 6, '0')
              elif (msg == "ออกจากการประเมิน"):
                 worksheet.update_cell(cell.row, 2, " ")
                 worksheet.update_cell(cell.row, 3, " ")
                 worksheet.update_cell(cell.row, 4, " ")
                 worksheet.update_cell(cell.row, 17, " ")
                 actual_status = '5'
                 Carousel=response_Carousel()
                 line_bot_api.reply_message(tk,Carousel)
              else:
                send_loop = "loop"
                actual_status = status_check
         elif (actual_status == '4'):
              if(msg == "ใช่" or msg =="เคย" or msg == "มี" or msg == "สูบ"):
                worksheet.update_cell(cell.row, 17, '4')
                worksheet.update_cell(cell.row, 7, '1')
              elif(msg == "ไม่ใช่" or msg =="ไม่เคย" or msg == "ไม่มี" or msg == "ไม่สูบ"or msg == "ไม่"):
                worksheet.update_cell(cell.row, 17, '4')
                worksheet.update_cell(cell.row, 7, '0')
              elif (msg == "ออกจากการประเมิน"):
                 worksheet.update_cell(cell.row, 2, " ")
                 worksheet.update_cell(cell.row, 3, " ")
                 worksheet.update_cell(cell.row, 4, " ")
                 worksheet.update_cell(cell.row, 17, " ")
                 actual_status = '5'
                 Carousel=response_Carousel()
                 line_bot_api.reply_message(tk,Carousel)
              else:
                send_loop = "loop"
                actual_status = status_check
         elif (actual_status == '5'):
              if(msg == "ใช่" or msg =="เคย" or msg == "มี" or msg == "สูบ"):
                worksheet.update_cell(cell.row, 17, '5')
                worksheet.update_cell(cell.row, 8, '1')
              elif(msg == "ไม่ใช่" or msg =="ไม่เคย" or msg == "ไม่มี" or msg == "ไม่สูบ"or msg == "ไม่"):
                worksheet.update_cell(cell.row, 17, '5')
                worksheet.update_cell(cell.row, 8, '0')
              elif (msg == "ออกจากการประเมิน" ):
                 worksheet.update_cell(cell.row, 2, " ")
                 worksheet.update_cell(cell.row, 3, " ")
                 worksheet.update_cell(cell.row, 4, " ")
                 worksheet.update_cell(cell.row, 17, " ")
                 actual_status = '5'

                 Carousel=response_Carousel()
                 line_bot_api.reply_message(tk,Carousel)
              else:
                send_loop = "loop"
                actual_status = status_check


    #print("actual_status",worksheet.cell(cell.row, 2).value)
    #print("user_id",user_id)
    #print("type_msg",type_msg)
        # เช็คว่า actual_status ตรงกับ expected_status หรือไม่

    cypher_query = (
            [f"MATCH (n)-[r]->(p) WHERE n.Assessdiabetes ='{type_msg}' and n.step ='{actual_status}' RETURN p.description,p.question,p.step,p.quickreply,p.finish"])
    with driver.session() as session:
        for Qcypher in cypher_query:
          result = session.run(Qcypher)
          for record in result:

            print("step",str(record['p.step']))
            print("actual_status_step",actual_status)
            #print ("finish",str(record['p.finish']))
            if actual_status == '1':#step2
                print("n1",actual_status)
                text_neo_ac = str(record['p.description'])+"\n"+str(record['p.question'])
                worksheet.update_cell(cell.row, 3, record["p.step"])
                #return str(record['p.description'])+"\n"+str(record['p.question']),estimate_quick_reply(cypher_query,line_bot_api,tk)
                return estimate_quick_reply(cypher_query,line_bot_api,tk,text_neo_ac,send_loop)

            elif actual_status == '2':#step3
                 text_neo_ac = str(record['p.question'])
                 worksheet.update_cell(cell.row, 3, record["p.step"])
                 return estimate_quick_reply(cypher_query,line_bot_api,tk,text_neo_ac,send_loop)
            elif actual_status == '3':#step4
                 worksheet.update_cell(cell.row, 3, record["p.step"])
                 text_neo_ac = str(record['p.question'])
                 return estimate_quick_reply(cypher_query,line_bot_api,tk,text_neo_ac,send_loop)
            elif actual_status == '4':#step5
                 text_neo_ac = str(record['p.question'])
                 worksheet.update_cell(cell.row, 3, record["p.step"])
                 return estimate_quick_reply(cypher_query,line_bot_api,tk,text_neo_ac,send_loop)
            elif actual_status == '5':#step6
                 worksheet.update_cell(cell.row, 3, record["p.step"])
                 worksheet.update_cell(cell.row, 2, " ")
                 worksheet.update_cell(cell.row, 3, " ")
                 worksheet.update_cell(cell.row, 4, " ")
                 worksheet.update_cell(cell.row, 17, " ")
                 send_loop = ""
                 print("send_loop",send_loop)
                 count_estimate  = int(worksheet.cell(cell.row, 5).value) + int(worksheet.cell(cell.row, 6).value) + int(worksheet.cell(cell.row, 7).value) +int(worksheet.cell(cell.row, 8).value)
                 print("count_estimate",count_estimate)
                 return estimate_count(count_estimate,type_msg,line_bot_api,tk,user_id)
                 #return str(record['p.description'])











In [ ]:
def estimate_quick_reply(Data,line_bot_api, tk,text_neo_ac,send_loop):
     data = []
     quick_reply_buttons = []
     my_cypher = Data
     with driver.session() as session:
            for Qcypher in my_cypher:
              result = session.run(Qcypher)
              for record in result:
                  if record['p.quickreply'] not in data:
                          data.append(record['p.quickreply'])
                          text_neo = (record['p.quickreply'])
                          print("text_neo",text_neo)
                          words = text_neo.split(",")
                          print("words",words)
                          text_neo_loop = str(record['p.question'])
                          quick_reply_button_1 = QuickReplyButton(
                                action=MessageAction(label=words[0], text=words[0])
                            )

                          quick_reply_button_2 = QuickReplyButton(
                                action=MessageAction(label=words[1], text=words[1])
                            )
                          quick_reply_button_3 = QuickReplyButton(
                                action=MessageAction(label="ออกจากการประเมิน", text="ออกจากการประเมิน")
                            )

                          quick_reply_buttons.append(quick_reply_button_1)
                          quick_reply_buttons.append(quick_reply_button_2)
                          quick_reply_buttons.append(quick_reply_button_3)

            if quick_reply_buttons and send_loop == "":
                quick_reply = QuickReply(items=quick_reply_buttons)
                return line_bot_api.reply_message(tk, [TextSendMessage(text=text_neo_ac,quick_reply=quick_reply)])
            elif quick_reply_buttons and send_loop != "":
                 text_loop = "ขณะนี้อยู่ในการประเมิน  กรุณาเลือกคำตอบให้ตรงกับช้อยที่กำหนด นะครับ \nหรือถ้าต้องการออกจากประเมิน กดปุ่มออก  ได้เลยครับ"
                 quick_reply = QuickReply(items=quick_reply_buttons)

                 return line_bot_api.reply_message(tk, [TextSendMessage(text=text_loop),TextSendMessage(text=text_neo_loop,quick_reply=quick_reply)])

In [ ]:
def estimate_count(count_estimate,type_msg,line_bot_api,tk,user_id):
     quick_reply_button = QuickReplyButton(
        action=MessageAction(label="เมนู", text="เมนู")
    )
     quick_reply = QuickReply(
        items=[quick_reply_button]
    )
     cell = worksheet.find(user_id)
     cypher_query = (
            [f"MATCH (n)-[r]->(p) WHERE n.Assessdiabetes ='{type_msg}' and n.finish ='1' and p.min <='{count_estimate}' and p.max>= '{count_estimate}'RETURN p.description"])
     with driver.session() as session:
        for Qcypher in cypher_query:
          result = session.run(Qcypher)
          for record in result:
              if(type_msg == "ประเมินโรคเบาหวาน"):
                text_neo = str(record['p.description'])
                worksheet.update_cell(cell.row, 9, count_estimate)
                return line_bot_api.reply_message(tk, TextSendMessage(text=text_neo,quick_reply=quick_reply))
              elif(type_msg == "ประเมินโรคความดันโลหิตสูง"):
                text_neo = str(record['p.description'])
                worksheet.update_cell(cell.row, 10, count_estimate)
                return line_bot_api.reply_message(tk, TextSendMessage(text=text_neo,quick_reply=quick_reply))
              elif(type_msg == "ประเมินโรคอ้วน"):
                text_neo = str(record['p.description'])
                worksheet.update_cell(cell.row, 11, count_estimate)
                return line_bot_api.reply_message(tk, TextSendMessage(text=text_neo,quick_reply=quick_reply))
              elif(type_msg == "ประเมินโรคตับแข็ง"):
                text_neo = str(record['p.description'])
                worksheet.update_cell(cell.row, 12, count_estimate)
                return line_bot_api.reply_message(tk, TextSendMessage(text=text_neo,quick_reply=quick_reply))
              elif(type_msg == "ประเมินโรคมะเร็ง"):
                text_neo = str(record['p.description'])
                worksheet.update_cell(cell.row, 13, count_estimate)
                return line_bot_api.reply_message(tk, TextSendMessage(text=text_neo,quick_reply=quick_reply))
              elif(type_msg == "ประเมินโรคหลอดเลือดสมอง"):
                text_neo = str(record['p.description'])
                worksheet.update_cell(cell.row, 14, count_estimate)
                return line_bot_api.reply_message(tk, TextSendMessage(text=text_neo,quick_reply=quick_reply))
              elif(type_msg == "ประเมินโรคสมองเสื่อม"):
                text_neo = str(record['p.description'])
                worksheet.update_cell(cell.row, 15, count_estimate)
                return line_bot_api.reply_message(tk, TextSendMessage(text=text_neo,quick_reply=quick_reply))
              elif(type_msg == "ประเมินโรคไตเรื้อรัง"):
                text_neo = str(record['p.description'])
                worksheet.update_cell(cell.row, 16, count_estimate)
                return line_bot_api.reply_message(tk, TextSendMessage(text=text_neo,quick_reply=quick_reply))
            #return str(record['p.description'])



In [ ]:
reply_msg("ประเมินโรคเบาหวาน","Ucc3e077cff6d7f39dc73f1ead7cc39e0","11","11")

In [ ]:
#def estimate_quick_reply(Data):
  #str_m = []
  #count_n = 0
  #data =[]
  #print("DATA",Data)
  #with driver.session() as session:
       # result = session.run(Data[0])
        #print("result",result)
        #for record in result:
         # if len(str(record['p.quickreply']))>0:
          #  if record['p.quickreply'] not in data:
           #   data.append(record['p.quickreply'])
           #   text = str(record['p.quickreply'])
           #   print("text:",text)
            #  words = text.split(",")
           #   for split_words in words:
            #      print(words)
              #    str_m.append({ "type": "action","imageUrl": "","action": { "type": "message", "label": split_words[:18]+".." , "text": split_words }})

       # return  {"items": str_m}

In [ ]:
def reply_msg(line_bot_api,tk,user_id,msg):
    #result = check_user_id(msg,user_id,line_bot_api,tk)
    check_user_id(line_bot_api,tk,user_id,msg)
    #text_msg = result[0]
    #quick_reply_dic = result[1]
    #text_result2 = result[2]
    #if (result[2]==2):
       # text_msg = text_msg
       # return text_msg,quick_reply_dic
    #else:
      #  return result,""

In [ ]:
def quick_reply_relationship(line_bot_api, tk, result):
   my_cypher = result
   data = []
   text_name =[]
   quick_reply_buttons = []
   with driver.session() as session:
            for Qcypher in my_cypher:
              result = session.run(Qcypher)
              for record in result:
                    msq_relation =str(record['r.name'])#มีลักษณะอาการ
                    if(str(record['p.description'])) !="None":
                      #text_message = str(record['type(r)'])+" "+msq_relation+" "+str(record['p.name'])+""+str(record['p.description'])
                      text_message = str(record['type(r)'])+str(record['r.name'])
                      text_name.append(record['p.name'])
                      print("text_name",text_name)
                      #data = str(record['p.name'])
                      if record['p.name'] not in data:
                          data.append(record['p.name'])
                          text_neo = str(record['p.name'])
                          quick_reply_button = QuickReplyButton(
                          action=MessageAction(label=text_neo[:18], text=text_neo)
                          )
                          quick_reply_buttons.append(quick_reply_button)
                    elif (str(record['p.description'])) =="None":
                          text_message = str(record['type(r)'])+str(record['r.name'])
                          text_name.append(record['p.name'])
              if quick_reply_buttons:
                  text_name_str = "-"+'\n-'.join(text_name)
                  quick_reply = QuickReply(items=quick_reply_buttons)
                  return line_bot_api.reply_message(tk, [TextSendMessage(text=text_message),TextSendMessage(text=text_name_str,quick_reply=quick_reply)])
              else:
                  text_name_str = "-"+'\n-'.join(text_name)
                  quick_reply = QuickReply(items=quick_reply_buttons)
                  return line_bot_api.reply_message(tk, [TextSendMessage(text=text_message),TextSendMessage(text=text_name_str)])



In [ ]:
reply_msg("อาการโรคเบาหวาน","Ucc3e077cff6d7f39dc73f1ead7cc39e0","11","11")

In [ ]:
def response_Carousel():

    notes = [
        CarouselColumn(
            thumbnail_image_url=
            "https://www.filesthaischool0.com/files/uppic/34111955/news/34111955_0_20230815-104743.png",
            title="สอบถามเพิ่มเติม",
            text="สามารถสอบคำถามทั่วไปเกี่ยวกับกลุ่มโรค NCDs",
            actions=[{
                "type": "message",
                "label": "สอบถาม",
                "text": "สอบถาม"
            },
            ]),
          CarouselColumn(
            thumbnail_image_url=
            "https://media.istockphoto.com/id/1265028866/vector/medical-checkup-icon-vector-for-web-design-book-magazine-poster-ads-application.jpg?s=612x612&w=0&k=20&c=e5zCSQ8tRbPA9rS2hWTbmNWGAUZGps5CZRoomTMqZfA=",
            title="ประเมิน",
            text="จะมีแบบสอบถามเกี่ยวกับโรคเพื่อประเมินความเสี่ยง",
            actions=[{
                "type": "message",
                "label": "ประเมิน",
                "text": "ประเมิน"
            },
            ]),
         CarouselColumn(
            thumbnail_image_url=
            "https://lh3.googleusercontent.com/d/1XsaWRTA-d413gHxPWfRRx9gloL6ADbvu",
            title="สมุนไพร",
            text="จะมีสมุนไพรที่เกี่ยวกับโรคสามารถเลือกดูได้เลย",
            actions=[{
                "type": "message",
                "label": "สมุนไพร",
                "text": "สมุนไพร"
            },
            ]),


    ]

    Carousel = TemplateSendMessage(
        alt_text='เมนู',
        template=CarouselTemplate(columns=notes),
    )
    return Carousel

In [ ]:
def response_flex():
  bubble_string = """
{
  "type": "bubble",
  "body": {
    "type": "box",
    "layout": "vertical",
    "spacing": "md",
    "contents": [
      {
        "wrap": true,
        "text": "สามารถเลือกโรคที่ต้องการประเมินได้เลยครับ 👇",
        "type": "text"
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "ประเมินโรคเบาหวาน",
          "text": "ประเมินโรคเบาหวาน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "ประเมินโรคความดันโลหิตสูง",
          "text": "ประเมินโรคความดันโลหิตสูง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "ประเมินโรคอ้วน",
          "text": "ประเมินโรคอ้วน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "ประเมินโรคตับแข็ง",
          "text": "ประเมินโรคตับแข็ง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "ประเมินโรคมะเร็ง",
          "text": "ประเมินโรคมะเร็ง"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "ประเมินโรคหลอดเลือดสมอง",
          "text": "ประเมินโรคหลอดเลือดสมอง"
        }
        },
       {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "ประเมินโรคสมองเสื่อม",
          "text": "ประเมินโรคสมองเสื่อม"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "ประเมินโรคไตเรื้อรัง",
          "text": "ประเมินโรคไตเรื้อรัง"
        }
      }
    ]
  }
}
"""

  flex = FlexSendMessage(alt_text="ประเมิน",contents=json.loads(bubble_string))
  return flex




In [ ]:
def symptom_flex():
  bubble_string = """
{
  "type": "bubble",
  "body": {
    "type": "box",
    "layout": "vertical",
    "spacing": "md",
    "contents": [
      {
        "wrap": true,
        "text": "สามารถเลือกอาการของโรคที่ต้องการได้เลย 👇",
        "type": "text"
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "อาการโรคเบาหวาน",
          "text": "อาการโรคเบาหวาน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "อาการโรคความดันโลหิตสูง",
          "text": "อาการความดันโลหิตสูง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "อาการโรคอ้วน",
          "text": "อาการโรคอ้วน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "อาการโรคตับแข็ง",
          "text": "อาการโรคตับแข็ง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "อาการโรคมะเร็ง",
          "text": "อาการโรคมะเร็ง"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "อาการโรคหลอดเลือดสมอง",
          "text": "อาการโรคหลอดเลือดสมอง"
        }
        },
       {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "อาการโรคสมองเสื่อม",
          "text": "อาการโรคสมองเสื่อม"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "อาการโรคไตเรื้อรัง",
          "text": "อาการโรคไตเรื้อรัง"
        }
      }
    ]
  }
}
"""

  flex = FlexSendMessage(alt_text="อาการ",contents=json.loads(bubble_string))
  return flex




In [ ]:
def cause_flex():
  bubble_string = """
{
  "type": "bubble",
  "body": {
    "type": "box",
    "layout": "vertical",
    "spacing": "md",
    "contents": [
      {
        "wrap": true,
        "text": "สามารถเลือกสาเหตุของโรคที่ต้องการได้เลย 👇",
        "type": "text"
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "สาเหตุของโรคเบาหวาน",
          "text": "สาเหตุของโรคเบาหวาน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "สาเหตุของโรคความดันโลหิตสูง",
          "text": "สาเหตุของโรคความดันโลหิตสูง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "สาเหตุของโรคอ้วน",
          "text": "สาเหตุของโรคอ้วน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "สาเหตุของโรคตับแข็ง",
          "text": "สาเหตุของโรคตับแข็ง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "สาเหตุของโรคมะเร็ง",
          "text": "สาเหตุของโรคมะเร็ง"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "สาเหตุของโรคหลอดเลือดสมอง",
          "text": "สาเหตุของโรคหลอดเลือดสมอง"
        }
        },
       {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "สาเหตุของโรคสมองเสื่อม",
          "text": "สาเหตุของโรคสมองเสื่อม"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "สาเหตุของโรคไตเรื้อรัง",
          "text": "สาเหตุของโรคไตเรื้อรัง"
        }
      }
    ]
  }
}
"""

  flex = FlexSendMessage(alt_text="สาเหตุ",contents=json.loads(bubble_string))
  return flex

In [ ]:
def protect_flex():
  bubble_string = """
{
  "type": "bubble",
  "body": {
    "type": "box",
    "layout": "vertical",
    "spacing": "md",
    "contents": [
      {
        "wrap": true,
        "text": "สามารถเลือกวิธีป้องกันของโรคที่ต้องการได้เลย 👇",
        "type": "text"
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "วิธีป้องกันของโรคเบาหวาน",
          "text": "วิธีป้องกันของโรคเบาหวาน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "วิธีป้องกันของโรคความดันโลหิตสูง",
          "text": "วิธีป้องกันของโรคความดันโลหิตสูง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "วิธีป้องกันของโรคอ้วน",
          "text": "วิธีป้องกันของโรคอ้วน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "วิธีป้องกันของโรคตับแข็ง",
          "text": "วิธีป้องกันของโรคตับแข็ง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "วิธีป้องกันของโรคมะเร็ง",
          "text": "วิธีป้องกันของโรคมะเร็ง"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "วิธีป้องกันของโรคหลอดเลือดสมอง",
          "text": "วิธีป้องกันของโรคหลอดเลือดสมอง"
        }
        },
       {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "วิธีป้องกันของโรคสมองเสื่อม",
          "text": "วิธีป้องกันของโรคสมองเสื่อม"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "วิธีป้องกันของโรคไตเรื้อรัง",
          "text": "วิธีป้องกันของโรคไตเรื้อรัง"
        }
      }
    ]
  }
}
"""

  flex = FlexSendMessage(alt_text="ป้องกัน",contents=json.loads(bubble_string))
  return flex

In [ ]:
def q_and_a():
  bubble_string = """
        {
  "type": "bubble",
  "body": {
    "type": "box",
    "layout": "vertical",
    "spacing": "md",
    "contents": [
      {
            "wrap": true,
            "text": "สามารถสอบถามทั่วไปตามไกด์ข้างล่างได้เลยครับ 👇",
            "type": "text"
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "โรคเบาหวานคืออะไร",
          "text": "โรคเบาหวานคืออะไร"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "อาการของโรคเบาหวานเกิดจากอะไร",
          "text": "อาการของโรคเบาหวานเกิดจากอะไร"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "NCDsคืออะไร",
          "text": "NCDsคืออะไร"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "NCDsมีโรคอะไรบ้าง",
          "text": "NCDsมีโรคอะไรบ้าง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#808080",
        "action": {
          "type": "message",
          "label": "เมนู",
          "text": "เมนู"
        }
      }
    ]
  }
}
        """
  flex = FlexSendMessage(alt_text="สอบถาม",contents=json.loads(bubble_string))
  return flex

In [ ]:
def herb_list():
    bubble_string = """
         {
  "type": "bubble",
  "body": {
    "type": "box",
    "layout": "vertical",
    "spacing": "md",
    "contents": [
      {
        "wrap": true,
        "text": "สามารถเลือกสมุนไพรตามโรคได้เลยครับ 👇",
        "type": "text"
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#82E0AA",
        "action": {
          "type": "message",
          "label": "สมุนไพรโรคเบาหวาน",
          "text": "สมุนไพรโรคเบาหวาน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#82E0AA",
        "action": {
          "type": "message",
          "label": "สมุนไพรโรคความดันโลหิตสูง",
          "text": "สมุนไพรโรคความดันโลหิตสูง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#82E0AA",
        "action": {
          "type": "message",
          "label": "สมุนไพรโรคอ้วน",
          "text": "สมุนไพรโรคอ้วน"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#82E0AA",
        "action": {
          "type": "message",
          "label": "สมุนไพรโรคตับแข็ง",
          "text": "สมุนไพรโรคตับแข็ง"
        }
      },
      {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#82E0AA",
        "action": {
          "type": "message",
          "label": "สมุนไพรโรคมะเร็ง",
          "text": "สมุนไพรโรคมะเร็ง"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#82E0AA",
        "action": {
          "type": "message",
          "label": "สมุนไพรโรคหลอดเลือดสมอง",
          "text": "สมุนไพรโรคหลอดเลือดสมอง"
        }
        },
       {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#82E0AA",
        "action": {
          "type": "message",
          "label": "สมุนไพรโรคสมองเสื่อม",
          "text": "สมุนไพรโรคสมองเสื่อม"
        }
        },
        {
        "type": "button",
        "style": "primary",
        "height": "sm",
        "margin": "sm",
        "color": "#82E0AA",
        "action": {
          "type": "message",
          "label": "สมุนไพรโรคไตเรื้อรัง",
          "text": "สมุนไพรโรคไตเรื้อรัง"
        }
      }
    ]
  }
}
    """
    herb = FlexSendMessage(alt_text="สมุนไพร",contents=json.loads(bubble_string))
    return herb

In [ ]:
def return_message(line_bot_api,tk,user_id,msg):


    if (msg == "เมนู" or msg == "menu" or msg == "Menu"):
        Carousel=response_Carousel()
        line_bot_api.reply_message(tk,Carousel)
    elif (msg == "ประเมิน"):

        flex = response_flex()
        line_bot_api.reply_message(tk,flex)
    elif (msg == "สอบถาม"):

          ask_question  = q_and_a()
          line_bot_api.reply_message(tk,ask_question)
    elif (msg == "สมุนไพร"):

         herb_list_ = herb_list()
         line_bot_api.reply_message(tk,herb_list_)

    elif (msg == "อาการ"):

         symptom_list_ = symptom_flex()
         line_bot_api.reply_message(tk,symptom_list_)
    elif (msg == "สาเหตุ"):

         cause_list_ = cause_flex()
         line_bot_api.reply_message(tk,cause_list_)
    elif (msg == "วิธีป้องกัน"):

         protect_list_ = protect_flex()
         line_bot_api.reply_message(tk,protect_list_)
    else:
      msg_reply = reply_msg(msg,user_id,line_bot_api,tk)
      line_bot_api.reply_message(tk,TextSendMessage(text=msg_reply[0],quick_reply=msg_reply[1]))



In [ ]:
def quick_reply_menu (line_bot_api,tk,user_id,msg):

    quick_reply_button = QuickReplyButton(
        action=MessageAction(label="เมนู", text="เมนู")
    )
    quick_reply = QuickReply(
        items=[quick_reply_button]
    )
    return line_bot_api.reply_message(tk, [TextSendMessage(quick_reply=quick_reply)])

In [ ]:

def greeting_reply(line_bot_api,tk,result):
   my_cypher = result
   print("my_cypher",result)
   data = []
   text_name =[]
   quick_reply_buttons = []
   with driver.session() as session:
            for Qcypher in my_cypher:
              result = session.run(Qcypher)
              for record in result:
                text_greeting = str(record['n.description'])
                text_greeting1 = str(record['n.arkname'])
                url_pattern = "https://lh3.googleusercontent.com/d/"+str(record['n.image'])
                image_url=ImageSendMessage(original_content_url=url_pattern, preview_image_url=url_pattern)
              print("url",image_url)
              quick_reply_button = QuickReplyButton(
                  action=MessageAction(label="เมนู", text="เมนู")
              )
              quick_reply = QuickReply(
                  items=[quick_reply_button]
              )

              return line_bot_api.reply_message(tk, [ image_url, TextSendMessage(text=text_greeting+text_greeting1, quick_reply=quick_reply)])



In [ ]:
reply_msg("สวัสดีครับ","Ucc3e077cff6d7f39dc73f1ead7cc39e0","11","11")

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *


import json

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    try:
        json_data = json.loads(body)
        access_token = '9mfCCWlnBGRerI71QYVkPimqooylc5T0MolaRYLbKJZZxuopqsp7BWH4YzDGgDkBGjISZnjhdZhcbRYDV77mmnZvpxkX/mZ7dn66V5+JQJF9EGA455n5gQyZBhuhs9DK3in8bmE+gszGubEJu5F+BwdB04t89/1O/w1cDnyilFU='
        secret = '97f2925506d4cd50c9645bfc04fe9428'
        line_bot_api = LineBotApi(access_token)
        handler = WebhookHandler(secret)
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)
        msg = json_data['events'][0]['message']['text']
        tk = json_data['events'][0]['replyToken']
        user_id = json_data['events'][0]['source']['userId']
        #msg_reply = reply_msg(msg,user_id)
        #line_bot_api.reply_message(tk,TextSendMessage(text=msg_reply[0],quick_reply=msg_reply[1]))
        reply_msg(line_bot_api,tk,user_id,msg)
        print(msg, tk)
        print("user_id",user_id)

    except:
        print(body)
    return 'OK'
if __name__ == "__main__":
  run_with_ngrok(app)
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://a985-34-125-230-30.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


<ipython-input-42-2ccfac39e256>:20: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-42-2ccfac39e256>:21: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(secret)
INFO:werkzeug:127.0.0.1 - - [14/Jan/2024 08:37:17] "POST / HTTP/1.1" 200 -


{"destination":"Uf71b9011a5c2ad615ac87975d3d66b26","events":[]}
frow 5
msg NCDsมีโรคอะไรบ้าง
เรียก entity_if
entity_score [0.66954374]
relation_score [0.7325649]
text_name ['โรคความดันโลหิตสูง']
text_name ['โรคความดันโลหิตสูง', 'โรคหลอดเลือดสมอง']
text_name ['โรคความดันโลหิตสูง', 'โรคหลอดเลือดสมอง', 'โรคสมองเสื่อม']
text_name ['โรคความดันโลหิตสูง', 'โรคหลอดเลือดสมอง', 'โรคสมองเสื่อม', 'โรคไตเรื้อรัง']
text_name ['โรคความดันโลหิตสูง', 'โรคหลอดเลือดสมอง', 'โรคสมองเสื่อม', 'โรคไตเรื้อรัง', 'โรคตับแข็ง']
text_name ['โรคความดันโลหิตสูง', 'โรคหลอดเลือดสมอง', 'โรคสมองเสื่อม', 'โรคไตเรื้อรัง', 'โรคตับแข็ง', 'โรคมะเร็ง']
text_name ['โรคความดันโลหิตสูง', 'โรคหลอดเลือดสมอง', 'โรคสมองเสื่อม', 'โรคไตเรื้อรัง', 'โรคตับแข็ง', 'โรคมะเร็ง', 'โรคอ้วน']
text_name ['โรคความดันโลหิตสูง', 'โรคหลอดเลือดสมอง', 'โรคสมองเสื่อม', 'โรคไตเรื้อรัง', 'โรคตับแข็ง', 'โรคมะเร็ง', 'โรคอ้วน', 'โรคเบาหวาน']


...

มะระขี้นก c5baf4fd76c94de1988bd84b7afc8f74
user_id Ucc3e077cff6d7f39dc73f1ead7cc39e0
frow 5
msg เมนู


INFO:werkzeug:127.0.0.1 - - [14/Jan/2024 08:45:11] "POST / HTTP/1.1" 200 -


เมนู 446f31786e3641f29930a55edb0749eb
user_id Ucc3e077cff6d7f39dc73f1ead7cc39e0
frow 5
msg อาการโรคเบาหวานมีอะไรบ้าง
เรียก entity_if
entity_score [0.89325917]
relation_score [0.90937304]
text_name ['ชาตามปลายนิ้วมือและนิ้วเท้า']
text_name ['ชาตามปลายนิ้วมือและนิ้วเท้า', 'ตาพร่ามัว']
text_name ['ชาตามปลายนิ้วมือและนิ้วเท้า', 'ตาพร่ามัว', 'น้ำหนักลด']
text_name ['ชาตามปลายนิ้วมือและนิ้วเท้า', 'ตาพร่ามัว', 'น้ำหนักลด', 'ปัสสาวะบ่อยขึ้นหิวน้ำมากขึ้น']
text_name ['ชาตามปลายนิ้วมือและนิ้วเท้า', 'ตาพร่ามัว', 'น้ำหนักลด', 'ปัสสาวะบ่อยขึ้นหิวน้ำมากขึ้น', 'อ่อนเพลีย']


INFO:werkzeug:127.0.0.1 - - [14/Jan/2024 08:45:30] "POST / HTTP/1.1" 200 -


อาการโรคเบาหวานมีอะไรบ้าง 786202843ba44c09b4918550139e825d
user_id Ucc3e077cff6d7f39dc73f1ead7cc39e0
